### Week 2, Day 2

We're about to create and use our own MCP Server and MCP Client!

It's pretty simple, but it's not super-simple. The excitment around MCP is about how easy it is to share and use other MCP Servers - making our own does involve a bit of work.

Let's review some python code made mostly by a hard-working Engineering Team:

accounts.py

In [31]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, set_default_openai_client
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown
from openai import AsyncAzureOpenAI

load_dotenv(override=True)

True

In [32]:
from accounts import Account

In [33]:
account = Account.get("Ed")
account

Account(name='ed', balance=9260.524, strategy='', holdings={'AMZN': 12}, transactions=[3 shares of AMZN at 81.162 each., 3 shares of AMZN at 23.046 each., 3 shares of AMZN at 50.1 each., 3 shares of AMZN at 92.184 each.], portfolio_value_time_series=[('2025-06-07 11:31:51', 9942.514), ('2025-06-07 11:31:54', 9921.514), ('2025-06-07 11:48:02', 9765.514), ('2025-06-07 11:48:45', 9819.514), ('2025-06-08 06:26:00', 10113.375999999998), ('2025-06-08 06:26:00', 10131.375999999998), ('2025-06-08 06:43:42', 10158.076), ('2025-06-08 06:43:44', 9600.076), ('2025-06-08 06:45:30', 9548.524), ('2025-06-08 06:45:30', 9980.524)])

In [34]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "ed", "balance": 9209.421999999999, "strategy": "", "holdings": {"AMZN": 15}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 81.162, "timestamp": "2025-06-07 11:31:51", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 23.046, "timestamp": "2025-06-08 06:25:59", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 50.1, "timestamp": "2025-06-08 06:43:42", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 92.184, "timestamp": "2025-06-08 06:45:30", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 17.034, "timestamp": "2025-06-08 07:08:31", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-06-07 11:31:51", 9942.514], ["2025-06-07 11:31:54", 9921.514], ["2025-06-0

In [35]:
account.report()

'{"name": "ed", "balance": 9209.421999999999, "strategy": "", "holdings": {"AMZN": 15}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 81.162, "timestamp": "2025-06-07 11:31:51", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 23.046, "timestamp": "2025-06-08 06:25:59", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 50.1, "timestamp": "2025-06-08 06:43:42", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 92.184, "timestamp": "2025-06-08 06:45:30", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 17.034, "timestamp": "2025-06-08 07:08:31", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-06-07 11:31:51", 9942.514], ["2025-06-07 11:31:54", 9921.514], ["2025-06-07 11:48:02", 9765.514], ["20

In [36]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 81.162,
  'timestamp': '2025-06-07 11:31:51',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 23.046,
  'timestamp': '2025-06-08 06:25:59',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 50.1,
  'timestamp': '2025-06-08 06:43:42',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 92.184,
  'timestamp': '2025-06-08 06:45:30',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 17.034,
  'timestamp': '2025-06-08 07:08:31',
  'rationale': 'Because this bookstore website looks promising'}]

### Now we write an MCP server and use it directly!

In [37]:
# Now let's use our accounts server as an MCP server

params = {"command": "/home/ateet/.local/bin/uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=10) as server:
    mcp_tools = await server.list_tools()


In [38]:
mcp_tools

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None),
 Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None),
 Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', '

In [39]:
import os

client = AsyncAzureOpenAI(
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),  
    api_version=os.getenv("AZURE_OPENAI_API_VERSION_LATEST"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)
set_default_openai_client(client)

instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Ed and my account is under the name Ed. What's my balance and my holdings?"
model = os.getenv("AZURE_DEPLOYMENT_NAME")

In [40]:


async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


Your current balance is **$9,209.42**. 

You hold **15 shares of AMZN (Amazon)** in your account.

### Now let's build our own MCP Client

In [47]:
import os
os.environ["PATH"] += os.pathsep + os.path.expanduser("~/.local/bin")

import shutil
print(shutil.which("uv"))


/home/ateet/.local/bin/uv


In [48]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None), Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None), Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', 'ty

In [49]:
agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
result = await Runner.run(agent, request)
display(Markdown(result.final_output))

Your account balance is **$9,209.42**. 

You hold **15 shares of AMZN** (Amazon). If you need any further assistance or have questions about your account, feel free to ask!

In [50]:
context = await read_accounts_resource("ed")
print(context)

{"name": "ed", "balance": 9209.421999999999, "strategy": "", "holdings": {"AMZN": 15}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 81.162, "timestamp": "2025-06-07 11:31:51", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 23.046, "timestamp": "2025-06-08 06:25:59", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 50.1, "timestamp": "2025-06-08 06:43:42", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 92.184, "timestamp": "2025-06-08 06:45:30", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 17.034, "timestamp": "2025-06-08 07:08:31", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-06-07 11:31:51", 9942.514], ["2025-06-07 11:31:54", 9921.514], ["2025-06-07 11:48:02", 9765.514], ["202

In [51]:
from accounts import Account
Account.get("ed").report()

'{"name": "ed", "balance": 9209.421999999999, "strategy": "", "holdings": {"AMZN": 15}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 81.162, "timestamp": "2025-06-07 11:31:51", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 23.046, "timestamp": "2025-06-08 06:25:59", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 50.1, "timestamp": "2025-06-08 06:43:42", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 92.184, "timestamp": "2025-06-08 06:45:30", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 17.034, "timestamp": "2025-06-08 07:08:31", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-06-07 11:31:51", 9942.514], ["2025-06-07 11:31:54", 9921.514], ["2025-06-07 11:48:02", 9765.514], ["20

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercises</h2>
            <span style="color:#ff7800;">Make your own MCP Server! Make a simple function to return the current Date, and expose it as a tool so that an Agent can tell you today's date.<br/>Harder optional exercise: then make an MCP Client, and use a native OpenAI call (without the Agents SDK) to use your tool via your client.
            </span>
        </td>
    </tr>
</table>